# Week 4 - Models and Experimentation

## Step 1 Training a model

For the purposes of this demo, we will be using this [adapted demo](https://www.datacamp.com/tutorial/xgboost-in-python) and training an XGBoost model, and then doing some experimentation and hyperparameter tuning.


If running this notebook locally, use the following steps to create virtual environment:
- Don't use past python 3.10
- To create virtual environment use "venv"

`python -m venv NAME`

- Try to avoid anaconda, poetry or similar package management platforms
- To install a package use pip

`python -m pip install <package-name>`

- once you are done working with this virtual environment, deactivate it with `deactivate`

### Install packages

In [1]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.0 MB/s eta 0:00:00


In [2]:
!pip install --upgrade wandb


In [3]:
import xgboost as xgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


### Import data

We will be using Diamonds dataset imported from Seaborn. It is also available on [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds).

Read about the features by following the link. We will be predicting the price of diamonds.

In [4]:
diamonds = sns.load_dataset('diamonds')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [5]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [6]:
diamonds.shape

(53940, 10)

In [7]:
X,y = diamonds.drop('price', axis=1), diamonds[['price']]

# For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.

X['cut'] = X['cut'].astype('category')
X['color'] = X['color'].astype('category')
X['clarity'] = X['clarity'].astype('category')

### Split the data and train a model

In [8]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [9]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:47:51] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [10]:
# Define evaluation metrics - Root Mean Squared Error

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

RMSE: 532.8838153117543


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:47:52] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


### Incorporate validation

In [11]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 100

# Create the validation set
evals = [(dtrain, "train"), (dtest, "validation")]

In [12]:
evals = [(dtrain, "train"), (dtest, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10,
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630
[10]	train-rmse:550.99470	validation-rmse:571.16640
[20]	train-rmse:491.51435	validation-rmse:544.08058


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:47:52] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[30]	train-rmse:464.38845	validation-rmse:537.01895
[40]	train-rmse:445.99106	validation-rmse:533.85127
[50]	train-rmse:430.36010	validation-rmse:532.90320
[60]	train-rmse:418.87898	validation-rmse:533.04629
[70]	train-rmse:409.66247	validation-rmse:533.58046
[80]	train-rmse:397.34048	validation-rmse:534.31963
[90]	train-rmse:389.94294	validation-rmse:532.61946
[99]	train-rmse:377.70831	validation-rmse:532.88383


In [13]:
# Incorporate early stopping
n = 10000


model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:47:53] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[50]	train-rmse:430.36010	validation-rmse:532.90320
[100]	train-rmse:377.56825	validation-rmse:532.79980
[103]	train-rmse:375.44970	validation-rmse:532.50220


In [14]:
# Cross-validation

params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 1000

results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:47:54] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [15]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2861.153015,8.266765,2861.773555,36.937516
1,2081.378004,5.534608,2084.973481,32.064109
2,1545.361682,3.287745,1553.681211,31.059209
3,1182.364236,3.585787,1192.464771,26.157805
4,941.828819,2.971779,958.467497,23.613538


In [16]:
best_rmse = results['test-rmse-mean'].min()

best_rmse

549.1039652582465

## Start W&B


- Login into your W&B profile using the code below
- Alternatively you can set environment variables. There are several env variables which you can set to change the behavior of W&B logging. The most important are:
    - WANDB_API_KEY - find this in your "Settings" section under your profile
    - WANDB_BASE_URL - this is the url of the W&B server

- Find your API Token in "Profile" -> "Setttings" in the W&B App



In [17]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [18]:
# TO DO
# Start experiment tracking with W&B
# Do at least 5 experiments with various hyperparameters
# Choose any method for hyperparameter tuning: grid search, random search, bayesian search
# Describe your findings and what you see

In [19]:
wandb.init()

wandb: Currently logged in as: chethanasaligram2025 (chethana). Use `wandb login --relogin` to force relogin


In [20]:
import random

# 5 simulated experiments
total_runs = 5
for run in range(total_runs):
  wandb.init(
      project="basic-intro",
      name=f"experiment_{run}",
      config={
      "learning_rate": 0.02,
      "architecture": "CNN",
      "dataset": "CIFAR-100",
      "epochs": 10,
      })

  epochs = 10
  offset = random.random() / 5
  for epoch in range(2, epochs):
      acc = 1 - 2 ** -epoch - random.random() / epoch - offset
      loss = 2 ** -epoch + random.random() / epoch + offset

      wandb.log({"acc": acc, "loss": loss})

  wandb.finish()

acc,▁▂▆▆██▇▇
loss,█▅▂▃▁▂▂▁
acc,0.75289
loss,0.2314


acc,▁▁▇▅██▇█
loss,█▆▅▅▄▁▁▁
acc,0.75848
loss,0.18356


acc,▁▁▄▇▅▇▆█
loss,█▃▅▃▂▂▁▁
acc,0.92555
loss,0.10788


acc,▁▃▆▆▆▇██
loss,▆█▃▅▄▂▁▁
acc,0.8297
loss,0.20657


acc,▁▃▆▆▆▇▇█
loss,▆█▃▂▄▃▁▁
acc,0.78612
loss,0.21967


In [21]:
#random search
wandb.init(project="hyperparameter-tuning", name="xgboost_experiments")
import random


def hyperparameter_tuning():
    for run in range(5):
        params = {
            "objective": "reg:squarederror",
            "tree_method": "gpu_hist",
            "learning_rate": random.uniform(0.01, 0.1),
            "max_depth": random.randint(3, 10),
            "subsample": random.uniform(0.5, 1.0),
            "colsample_bytree": random.uniform(0.5, 1.0),
            "gamma": random.uniform(0, 1),
            "min_child_weight": random.uniform(1, 10)
        }

        # Create the training and validation sets
        # dtrain = xgb.DMatrix(X_train, label=y_train)
        # dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)

        # dtest = xgb.DMatrix(X_test, label=y_test)

        evals = [(dtrain, "train"), (dtest, "validation")]

        model = xgb.train(
            params=params,
            dtrain=dtrain,
            num_boost_round=1000,
            evals=evals,
            early_stopping_rounds=50,
            verbose_eval=50
        )

        predictions = model.predict(dtest)
        rmse = mean_squared_error(y_test, predictions, squared=False)

        wandb.log({"run": run, "hyperparameters": params, "RMSE": rmse})

hyperparameter_tuning()

wandb.finish()

[0]	train-rmse:3711.78868	validation-rmse:3710.20878


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:55:06] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[50]	train-rmse:692.35193	validation-rmse:691.37568
[100]	train-rmse:570.40782	validation-rmse:581.79273
[150]	train-rmse:541.79099	validation-rmse:565.09245
[200]	train-rmse:524.50089	validation-rmse:556.45861
[250]	train-rmse:512.76573	validation-rmse:555.28596
[300]	train-rmse:502.91899	validation-rmse:552.44625
[350]	train-rmse:493.24436	validation-rmse:549.64707
[400]	train-rmse:485.85219	validation-rmse:547.26897
[450]	train-rmse:479.61487	validation-rmse:546.58419
[473]	train-rmse:476.75669	validation-rmse:547.02273
[0]	train-rmse:3904.89829	validation-rmse:3902.03952


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:55:08] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:55:08] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[50]	train-rmse:1449.08468	validation-rmse:1443.89438
[100]	train-rmse:719.95124	validation-rmse:728.57138
[150]	train-rmse:546.71472	validation-rmse:572.27921
[200]	train-rmse:502.69480	validation-rmse:539.54479
[250]	train-rmse:484.53812	validation-rmse:531.24729
[300]	train-rmse:472.90018	validation-rmse:529.36112
[350]	train-rmse:463.94325	validation-rmse:527.95456
[400]	train-rmse:456.61617	validation-rmse:527.39180
[450]	train-rmse:450.01487	validation-rmse:527.55247
[464]	train-rmse:447.84681	validation-rmse:527.70542
[0]	train-rmse:3984.80849	validation-rmse:3981.99545


...

[50]	train-rmse:582.16593	validation-rmse:610.45738
[100]	train-rmse:459.39342	validation-rmse:528.42689
[150]	train-rmse:430.58162	validation-rmse:527.50321
[176]	train-rmse:420.33390	validation-rmse:528.13167
[0]	train-rmse:3942.44299	validation-rmse:3939.67609
[50]	train-rmse:2255.86010	validation-rmse:2248.97833


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:55:15] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:55:15] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[100]	train-rmse:1427.80504	validation-rmse:1421.02208
[150]	train-rmse:1040.21021	validation-rmse:1034.14865
[200]	train-rmse:853.11323	validation-rmse:848.23925
[250]	train-rmse:754.08255	validation-rmse:750.93572
[300]	train-rmse:694.55821	validation-rmse:694.97257
[350]	train-rmse:654.66000	validation-rmse:656.93096
[400]	train-rmse:624.02161	validation-rmse:628.37112
[450]	train-rmse:602.45194	validation-rmse:608.71585
[500]	train-rmse:586.22459	validation-rmse:594.11794
[550]	train-rmse:573.37596	validation-rmse:582.48092
[600]	train-rmse:564.74614	validation-rmse:575.11699
[650]	train-rmse:557.14929	validation-rmse:569.31489
[700]	train-rmse:550.31597	validation-rmse:564.04352
[750]	train-rmse:544.87038	validation-rmse:560.08674
[800]	train-rmse:540.49212	validation-rmse:557.33425
[850]	train-rmse:536.04969	validation-rmse:554.35959
[900]	train-rmse:532.37804	validation-rmse:552.46130
[950]	train-rmse:529.00440	validation-rmse:550.70384
[999]	train-rmse:526.07168	validation-rmse

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:55:19] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


RMSE,▇▁▄▁█
run,▁▃▅▆█
RMSE,549.34934
run,4


Experiments -

Random Search: In this experiment, I have employed random search for hyperparameter tuning of an XGBoost model using the Weights and Biases (W&B) experiment tracking framework. Random search involves randomly sampling hyperparameter values within predefined ranges and evaluating their performance. The hyperparameters considered include learning rate, max depth, subsample ratio, colsample by tree ratio, gamma and min child weight. For each randomly sampled set of hyperparameters, I have trained the XGBoost model using GPU acceleration and logged the resulting root mean squared error (RMSE) metric to the W&B dashboard. By analyzing the RMSE values across different hyperparameter configurations, we can identify the combination that yields the lowest error and thus the best model performance.

In [22]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}


In [ ]:
#Grid search first try
import itertools

param_grid = {
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 5, 7],
    "subsample": [0.5, 0.75, 1.0],
    "colsample_bytree": [0.5, 0.75, 1.0],
    "gamma": [0, 0.5, 1],
    "min_child_weight": [1, 5, 10]
}

param_combinations = itertools.product(*param_grid.values())

wandb.init(project="hyperparameter-tuning", name="xgboost_grid_search")

def hyperparameter_tuning():
    for idx, params in enumerate(param_combinations):
        param_dict = {param_name: str(param_value) for param_name, param_value in zip(param_grid.keys(), params)}

        param_dict["objective"] = "reg:squarederror"
        param_dict["tree_method"] = "gpu_hist"

        dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
        dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

        evals = [(dtrain, "train"), (dtest, "validation")]

        model = xgb.train(
            params=param_dict,
            dtrain=dtrain,
            num_boost_round=1000,
            evals=evals,
            early_stopping_rounds=50,
            verbose_eval=50
        )

        predictions = model.predict(dtest)
        rmse = mean_squared_error(y_test, predictions, squared=False)

        wandb.log({"run": idx,
                   "hyperparameters": param_dict,
                   "RMSE": rmse,
                   "x-axis": "Hyperparameter Combination",
                   "y-axis": "RMSE"})

hyperparameter_tuning()

wandb.finish()


RMSE,████▆▆▆▅▆▆▆███▆▆▆▆▅▅▅████▆▆▆▆▆▆▆▂▂▂▁▁▁▁▁
run,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
RMSE,534.13111
run,101
x-axis,Hyperparameter Combi...
y-axis,RMSE


[0]	train-rmse:3987.38250	validation-rmse:3984.68006


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [19:11:28] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[50]	train-rmse:2661.76612	validation-rmse:2651.54872
[100]	train-rmse:1913.63512	validation-rmse:1901.45932
[150]	train-rmse:1490.86364	validation-rmse:1479.71401
[200]	train-rmse:1239.40295	validation-rmse:1228.59808
[250]	train-rmse:1092.35540	validation-rmse:1082.02017
[300]	train-rmse:999.97437	validation-rmse:989.54880
[350]	train-rmse:929.69087	validation-rmse:919.93874
[400]	train-rmse:874.33891	validation-rmse:864.88212
[450]	train-rmse:835.53898	validation-rmse:826.56438
[500]	train-rmse:801.41464	validation-rmse:793.11289
[550]	train-rmse:770.26482	validation-rmse:762.76211
[600]	train-rmse:744.38220	validation-rmse:738.03813
[650]	train-rmse:726.95866	validation-rmse:721.61544
[700]	train-rmse:708.10230	validation-rmse:703.96868
[750]	train-rmse:691.55948	validation-rmse:688.13158
[800]	train-rmse:678.61909	validation-rmse:676.45956
[850]	train-rmse:667.80042	validation-rmse:666.55351
[900]	train-rmse:658.66178	validation-rmse:658.20634
[950]	train-rmse:649.06086	validation

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [19:11:31] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [19:11:31] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


...

[100]	train-rmse:1846.36082	validation-rmse:1834.60915
[150]	train-rmse:1414.33303	validation-rmse:1403.75162
[200]	train-rmse:1167.69936	validation-rmse:1156.25826
[250]	train-rmse:1024.75986	validation-rmse:1012.53004
[300]	train-rmse:934.20894	validation-rmse:921.25854
[350]	train-rmse:868.92122	validation-rmse:856.58845
[400]	train-rmse:815.74689	validation-rmse:803.55343
[450]	train-rmse:776.28099	validation-rmse:765.16367
[500]	train-rmse:744.59201	validation-rmse:734.39356
[550]	train-rmse:718.91923	validation-rmse:710.15182
[600]	train-rmse:698.37289	validation-rmse:690.92055
[650]	train-rmse:681.92368	validation-rmse:675.39489
[700]	train-rmse:667.21233	validation-rmse:661.49639
[750]	train-rmse:654.55475	validation-rmse:649.43764
[800]	train-rmse:644.74985	validation-rmse:640.56159
[850]	train-rmse:635.60424	validation-rmse:631.83148
[900]	train-rmse:628.45655	validation-rmse:625.26222
[950]	train-rmse:621.05980	validation-rmse:618.44759
[999]	train-rmse:615.80711	validation-

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [19:12:19] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [19:12:19] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[100]	train-rmse:1829.36868	validation-rmse:1817.27986
[150]	train-rmse:1393.60763	validation-rmse:1381.77370
[200]	train-rmse:1138.92105	validation-rmse:1125.35052
[250]	train-rmse:992.62876	validation-rmse:977.93043
[300]	train-rmse:900.77521	validation-rmse:886.23228
[350]	train-rmse:838.31300	validation-rmse:824.70100
[400]	train-rmse:793.22192	validation-rmse:779.95880
[450]	train-rmse:754.93196	validation-rmse:743.79081
[500]	train-rmse:724.28085	validation-rmse:714.50474
[550]	train-rmse:699.45424	validation-rmse:690.96383
[600]	train-rmse:679.40196	validation-rmse:672.63351
[650]	train-rmse:663.50026	validation-rmse:657.88489
[700]	train-rmse:650.01836	validation-rmse:645.59669
[750]	train-rmse:638.72708	validation-rmse:635.06787
[800]	train-rmse:629.22046	validation-rmse:626.84114
[850]	train-rmse:621.16408	validation-rmse:620.11682
[900]	train-rmse:614.03653	validation-rmse:614.19963
[950]	train-rmse:607.44200	validation-rmse:608.42178
[999]	train-rmse:602.37232	validation-rm

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [19:12:22] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [19:12:22] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[100]	train-rmse:1829.93846	validation-rmse:1818.03536


Experiments -

Grid Search:
In this experiment, I have employed grid search for hyperparameter tuning of an XGBoost model using the Weights and Biases (W&B) experiment tracking framework. Grid search systematically explores all possible combinations of hyperparameter values specified within predefined ranges. The hyperparameters considered include learning rate, max depth, subsample ratio, colsample by tree ratio, gamma and min child weight. I have defined a grid of parameter values for each hyperparameter and iterate through all combinations. For each combination, we train the XGBoost model using GPU acceleration and log the resulting root mean squared error (RMSE) metric to the W&B dashboard. By visualizing the RMSE values across different hyperparameter combinations, we can identify the optimal set of hyperparameters that minimizes the error and maximizes model performance.